# Notebook for extracting gold information from match timelines from database

## Script for extraction

### init mongo

In [ ]:
%pip install pymongo

from pymongo import MongoClient

In [2]:
client = MongoClient("localhost", 27017)

db = client.league_of_legends
db_match_timelines = db.match_timelines
db_matches_gold = db.matches_gold

### script

In [9]:
gold_differences = []

for match_timeline in db_match_timelines.find():
    minute_15_frame = min(15, len(match_timeline['info']['frames']) - 1)
    team_1_gold = 0
    team_2_gold = 0

    participant_number = 1
    for participant in match_timeline['info']['frames'][minute_15_frame]['participantFrames'].values():
        if participant_number <= 5:
            team_1_gold += int(participant['totalGold'])
        else:
            team_2_gold += int(participant['totalGold'])
        
        participant_number += 1

    winning_team = match_timeline['info']['frames'][-1]['events'][-1]['winningTeam']

    gold_difference = 0
    if winning_team == 100:
        gold_difference = team_1_gold - team_2_gold
    else:
        gold_difference = team_2_gold - team_1_gold

    gold_differences.append(gold_difference)

In [10]:
for gold_difference in gold_differences:
    db_matches_gold.insert_one({"gold_difference": gold_difference})